In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import os
from typing import List

In [2]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
# from sqlalchemy import Column, Date, Integer, String, Float
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.dialects.sqlite import insert

## Create database

In [3]:
# create database, tables, and indexes
def create_database():

    PROJECT_SRC = '/workspace/src'
    os.chdir(PROJECT_SRC)

    SQLALCHEMY_DATABASE_URI='sqlite:///../data/database.db'
    engine = create_engine(SQLALCHEMY_DATABASE_URI, echo=False)

    with engine.connect() as conn:

        gen_table='''
        CREATE TABLE IF NOT EXISTS GENERATION (
        dttm TEXT PRIMARY KEY, 
        wind REAL,
        solar REAL
        );'''
        conn.execute(gen_table)

        gen_idx='''
        CREATE UNIQUE INDEX IF NOT EXISTS GENERATION_IDX
        ON GENERATION(dttm);
        '''
        conn.execute(gen_idx)


        load_table='''
        CREATE TABLE IF NOT EXISTS LOAD (
        dttm TEXT PRIMARY KEY, 
        load_mwh REAL
        );'''
        conn.execute(load_table)

        load_idx='''
        CREATE UNIQUE INDEX IF NOT EXISTS LOAD_IDX
        ON LOAD(dttm);
        '''
        conn.execute(load_idx)

        price_table='''
        CREATE TABLE IF NOT EXISTS PRICE (
        dttm TEXT PRIMARY KEY, 
        price REAL
        );'''
        conn.execute(price_table)

        price_idx='''
        CREATE UNIQUE INDEX IF NOT EXISTS PRICE_IDX
        ON PRICE(dttm);
        '''
        conn.execute(price_idx)
        
    return engine
    
    

In [4]:
engine = create_database()

In [5]:
engine.table_names()

/tmp/ipykernel_182/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['GENERATION', 'LOAD', 'PRICE']

In [6]:
# url examples

# hourly forecasted and actual load by region, starts 20200101
# loads stay in standard time
# https://docs.misoenergy.org/marketreports/20210314_rf_al.xls
# https://docs.misoenergy.org/marketreports/20211107_rf_al.xls

# hourly generation mix by region, starts 20200101
# https://docs.misoenergy.org/marketreports/20210214_sr_gfm.xlsx
# hourly region price report, starts 20200101
# https://docs.misoenergy.org/marketreports/20220603_rt_pr.xls



# base url for imports
base_url = 'https://docs.misoenergy.org/marketreports/'
load_file = '_rf_al.xls'
generation_file = '_sr_gfm.xlsx'
price_file = '_rt_pr.xls'

In [7]:
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2022, 6, 1)
days = pd.date_range(start, end)
days

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26',
               '2022-05-27', '2022-05-28', '2022-05-29', '2022-05-30',
               '2022-05-31', '2022-06-01'],
              dtype='datetime64[ns]', length=883, freq='D')

In [8]:
# get YYYYMMDD string from date
days_str = [str(d).split(' ')[0].replace('-','') for d in days]
days_str[0:10]

['20200101',
 '20200102',
 '20200103',
 '20200104',
 '20200105',
 '20200106',
 '20200107',
 '20200108',
 '20200109',
 '20200110']

## load data

In [9]:
def get_load_data(
    days_idx: int, 
    base_url: str = base_url, 
    days_str: List[str] = days_str, 
    file: str = load_file,
):
    # build url and fetch data
    url = base_url + days_str[days_idx] + file
    load_data = pd.read_excel(url, header=1, skiprows=4).iloc[1:26,1:]
    
    # remove rows with missing hour ending values and rename columns
    load_data = load_data[~load_data.HourEnding.isna()]
    load_data = load_data[['Market Day', 'HourEnding', 'North ActualLoad (MWh)']]
    load_data = load_data.rename(
        columns={'Market Day':'day',
                'HourEnding':'he',
                'North ActualLoad (MWh)':'load_mwh'}
    )
    
    # convert data types
    load_data['he'] = load_data.he.astype(int)
    load_data['load_mwh'] = load_data.load_mwh.astype(float)
    load_data.day= pd.to_datetime(load_data.day)
    
    # create dttm index
    load_data['dttm'] = (
        load_data.day + 
        pd.to_timedelta(load_data.he, unit='h')
    )
    
    load_data = load_data[['dttm', 'load_mwh']].set_index('dttm')
    
    return load_data

In [10]:
load_data = get_load_data(0)
load_data

,load_mwh
dttm,
2019-12-31 01:00:00,16498.05
2019-12-31 02:00:00,15957.53
2019-12-31 03:00:00,15625.18
2019-12-31 04:00:00,15465.22
2019-12-31 05:00:00,15441.70
2019-12-31 06:00:00,15690.70
2019-12-31 07:00:00,16354.92
2019-12-31 08:00:00,17328.64
2019-12-31 09:00:00,18136.55


In [11]:
load_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2019-12-31 01:00:00 to 2020-01-01 00:00:00
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   load_mwh  24 non-null     float64
dtypes: float64(1)
memory usage: 384.0 bytes


## generation data

In [12]:
def get_generation_data(
    days_idx: int, 
    base_url: str = base_url, 
    days_str: List[str] = days_str, 
    file: str = generation_file,
):
    # build url and fetch data
    url = base_url + days_str[days_idx] + file
    generation_data = pd.read_excel(url, skiprows=2, header=[1,2])
    
    # save he data
    # he = hour ending
    he = generation_data[('Unnamed: 0_level_0', 'Market Hour Ending')]
    
    # get wind, solar, other generation from north region
    generation_data = generation_data['North']
    cols = [c for c in generation_data.columns if c in ['Wind', 'Solar', 'Other']]
    generation_data = generation_data[cols]
    
    # set he data and remove non numeric values
    generation_data['he'] = he
    idx = generation_data.he.str.isnumeric() != False
    generation_data = generation_data[idx]
    
    # if solar generation was not reported estimate generation from other
    # by removing subtracting off night generation
    if 'Solar' not in generation_data.columns:
        generation_data['Solar'] = (generation_data.Other - 
                                    np.max((generation_data.Other[:5].max(), generation_data.Other[-5:].max()))
                                   )
        generation_data.loc[generation_data.Solar < 0, 'Solar'] = 0
    
    generation_data.drop('Other', axis=1, inplace=True)
    
    # create dttm index
    generation_data['day'] = pd.to_datetime(days_str[days_idx])
    
    generation_data['dttm'] = (
        generation_data.day + 
        pd.to_timedelta(generation_data.he, unit='h')
    )
    generation_data = generation_data.drop(['he', 'day'], axis=1).set_index('dttm')
    
    return generation_data

In [13]:
generation_data = get_generation_data(0)
generation_data

/opt/conda/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Wind,Solar
dttm,,
2020-01-01 01:00:00,12443.0,0.0
2020-01-01 02:00:00,12338.8,0.0
2020-01-01 03:00:00,12075.4,0.0
2020-01-01 04:00:00,11957.9,0.0
2020-01-01 05:00:00,12012.6,0.0
2020-01-01 06:00:00,11992.1,0.0
2020-01-01 07:00:00,11709.8,0.0
2020-01-01 08:00:00,11186.4,0.0
2020-01-01 09:00:00,10625.5,0.0


In [14]:
generation_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2020-01-01 01:00:00 to 2020-01-02 00:00:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Wind    24 non-null     float64
 1   Solar   24 non-null     float64
dtypes: float64(2)
memory usage: 576.0 bytes


## price data

In [15]:
def get_price_data(
    days_idx: int, 
    base_url: str = base_url, 
    days_str: List[str] = days_str, 
    file: str = price_file,
):
    # build url and fetch data
    url = base_url + days_str[days_idx] + file
    price_data = pd.read_excel(url, skiprows=11, header=0)
    
    # will use minnesota hub prices
    # he = hour ending
    price_data = price_data.rename(columns={'Unnamed: 0':'he', 'Minnesota Hub':'price'})
    
    # get only he row containing Hour
    idx = ['Hour' in he for he in price_data.he]
    price_data = price_data.loc[idx, ['he', 'price']]
    
    # set day and get he as int
    price_data['day'] = pd.to_datetime(days_str[days_idx])
    price_data['he'] = [int(he.split()[1]) for he in price_data.he]
    
    # get dttm and drop day and he
    price_data['dttm'] = (
        price_data.day + 
        pd.to_timedelta(price_data.he, unit='h')
    )
    price_data = price_data.drop(['he', 'day'], axis=1).set_index('dttm')
    
    return price_data

In [16]:
price_data = get_price_data(0)
price_data

,price
dttm,
2020-01-01 01:00:00,12.52
2020-01-01 02:00:00,12.35
2020-01-01 03:00:00,12.06
2020-01-01 04:00:00,11.97
2020-01-01 05:00:00,12.37
2020-01-01 06:00:00,14.66
2020-01-01 07:00:00,15.67
2020-01-01 08:00:00,14.46
2020-01-01 09:00:00,15.55


In [17]:
price_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2020-01-01 01:00:00 to 2020-01-02 00:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   24 non-null     float64
dtypes: float64(1)
memory usage: 384.0 bytes


## Test upserts

### Generation

In [18]:
def upsert_generation(
    generation_data: pd.core.frame.DataFrame, 
    engine: sqlalchemy.engine.base.Engine,
):
    
    with engine.connect() as conn:

        generation_data.to_sql('GENERATION_TMP', engine, if_exists='replace')

        upsert='''INSERT INTO GENERATION 
        SELECT *
        FROM GENERATION_TMP WHERE true
        ON CONFLICT (dttm) DO UPDATE SET Wind=excluded.Wind, Solar=excluded.Solar;
        '''

        conn.execute(upsert)

        drop_tbl='DROP TABLE GENERATION_TMP'
        conn.execute(drop_tbl)

In [19]:
upsert_generation(generation_data, engine)
upsert_generation(generation_data, engine)

In [20]:
type(generation_data)

pandas.core.frame.DataFrame

In [21]:
type(engine)

sqlalchemy.engine.base.Engine

In [22]:
# engine.table_names()

In [23]:
with engine.connect() as conn:
    
    sql_text='''
    SELECT *
    FROM GENERATION ;
    '''
    
    res = pd.read_sql(sql_text, engine)
    
res

,dttm,wind,solar
0,2020-01-01 01:00:00.000000,12443.0,0.0
1,2020-01-01 02:00:00.000000,12338.8,0.0
2,2020-01-01 03:00:00.000000,12075.4,0.0
3,2020-01-01 04:00:00.000000,11957.9,0.0
4,2020-01-01 05:00:00.000000,12012.6,0.0
...,...,...,...
21187,2022-06-01 20:00:00.000000,11555.5,28.4
21188,2022-06-01 21:00:00.000000,10983.1,3.8
21189,2022-06-01 22:00:00.000000,10446.7,0.5
21190,2022-06-01 23:00:00.000000,10249.1,0.2


### Load

In [24]:
def upsert_load(
    load_data: pd.core.frame.DataFrame, 
    engine: sqlalchemy.engine.base.Engine,
):
    
    with engine.connect() as conn:

        load_data.to_sql('LOAD_TMP', engine, if_exists='replace')

        upsert='''INSERT INTO LOAD 
        SELECT *
        FROM LOAD_TMP WHERE true
        ON CONFLICT (dttm) DO UPDATE SET load_mwh=excluded.load_mwh;
        '''

        conn.execute(upsert)

        drop_tbl='DROP TABLE LOAD_TMP'
        conn.execute(drop_tbl)

In [25]:
upsert_load(load_data, engine)
upsert_load(load_data, engine)

In [26]:
# engine.table_names()

In [27]:
with engine.connect() as conn:
    
    sql_text='''
    SELECT *
    FROM LOAD ;
    '''
    
    res = pd.read_sql(sql_text, engine)
    
res

,dttm,load_mwh
0,2019-12-31 01:00:00.000000,16498.05
1,2019-12-31 02:00:00.000000,15957.53
2,2019-12-31 03:00:00.000000,15625.18
3,2019-12-31 04:00:00.000000,15465.22
4,2019-12-31 05:00:00.000000,15441.70
...,...,...
21187,2022-05-31 20:00:00.000000,18099.66
21188,2022-05-31 21:00:00.000000,17740.72
21189,2022-05-31 22:00:00.000000,17338.54
21190,2022-05-31 23:00:00.000000,16304.94


### Prices

In [28]:
def upsert_prices(
    price_data: pd.core.frame.DataFrame, 
    engine: sqlalchemy.engine.base.Engine,
):
    
    with engine.connect() as conn:

        price_data.to_sql('PRICE_TMP', engine, if_exists='replace')

        upsert='''INSERT INTO PRICE 
        SELECT *
        FROM PRICE_TMP WHERE true
        ON CONFLICT (dttm) DO UPDATE SET price=excluded.price;
        '''

        conn.execute(upsert)

        drop_tbl='DROP TABLE PRICE_TMP'
        conn.execute(drop_tbl)
    

In [29]:
upsert_prices(price_data, engine)
upsert_prices(price_data, engine)

In [30]:
# engine.table_names()

In [31]:
with engine.connect() as conn:
    
    sql_text='''
    SELECT *
    FROM PRICE ;
    '''
    
    res = pd.read_sql(sql_text, engine)
    
res

,dttm,price
0,2020-01-01 01:00:00.000000,12.52
1,2020-01-01 02:00:00.000000,12.35
2,2020-01-01 03:00:00.000000,12.06
3,2020-01-01 04:00:00.000000,11.97
4,2020-01-01 05:00:00.000000,12.37
...,...,...
21187,2022-06-01 20:00:00.000000,124.15
21188,2022-06-01 21:00:00.000000,58.47
21189,2022-06-01 22:00:00.000000,93.07
21190,2022-06-01 23:00:00.000000,40.10


## Combine data

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [34]:
for i in range(10):
    print('\n_____________________________')
    print(f'working on day: {days_str[i]}')
    
    print('\t-getting load...')
    # filename = '../data/load_data.csv'
    # hdr = False  if os.path.isfile(filename) else True
    load_data = get_load_data(i)
    upsert_load(load_data, engine)
    # load_data.to_csv(filename, mode='a', header=hdr)
    print(f'\t\tload_data shape: {load_data.shape}')
    
    print('\t-getting generation...')
    # filename = '../data/generation_data.csv'
    # hdr = False  if os.path.isfile(filename) else True
    generation_data = get_generation_data(i)
    upsert_generation(generation_data, engine)
    # generation_data.to_csv(filename, mode='a', header=hdr)
    print(f'\t\tgeneration_data shape: {generation_data.shape}')
    
    print('\t-getting prices...')
    # filename = '../data/price_data.csv'
    # hdr = False  if os.path.isfile(filename) else True
    price_data = get_price_data(i)
    upsert_prices(price_data, engine)
    # price_data.to_csv(filename)
    print(f'\t\tprice_data shape: {price_data.shape}')
    
    time.sleep(1)


_____________________________
working on day: 20200101
	-getting load...
		load_data shape: (24, 1)
	-getting generation...
		generation_data shape: (24, 2)
	-getting prices...
		price_data shape: (24, 1)

_____________________________
working on day: 20200102
	-getting load...
		load_data shape: (24, 1)
	-getting generation...
		generation_data shape: (24, 2)
	-getting prices...
		price_data shape: (24, 1)

_____________________________
working on day: 20200103
	-getting load...
		load_data shape: (24, 1)
	-getting generation...
		generation_data shape: (24, 2)
	-getting prices...
		price_data shape: (24, 1)

_____________________________
working on day: 20200104
	-getting load...
		load_data shape: (24, 1)
	-getting generation...
		generation_data shape: (24, 2)
	-getting prices...
		price_data shape: (24, 1)

_____________________________
working on day: 20200105
	-getting load...
		load_data shape: (24, 1)
	-getting generation...
		generation_data shape: (24, 2)
	-getting prices..

In [36]:
with engine.connect() as conn:
    
    sql_text='''
    SELECT *
    FROM PRICE ;
    '''
    
    res = pd.read_sql(sql_text, engine)
    
res

,dttm,price
0,2020-01-01 01:00:00.000000,12.52
1,2020-01-01 02:00:00.000000,12.35
2,2020-01-01 03:00:00.000000,12.06
3,2020-01-01 04:00:00.000000,11.97
4,2020-01-01 05:00:00.000000,12.37
...,...,...
21187,2022-06-01 20:00:00.000000,124.15
21188,2022-06-01 21:00:00.000000,58.47
21189,2022-06-01 22:00:00.000000,93.07
21190,2022-06-01 23:00:00.000000,40.10
